# Team 09 - Milestone 5

## Import libraries

In [ ]:
import requests
import imutils

# import pyfirmata as pyf
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
# import argparse
import skimage
# from pyfirmata import  Arduino ,SERVO, util
from time import sleep


## Arduino Connection

In [ ]:
board = pyf.Arduino('/dev/ttyACM2')
#the port on ubuntu 
it = pyf.util.Iterator(board)
it.start()
#identifying serial between arduino and python 

pin = 8 
#identifying pin
board.digital[pin].mode=SERVO

# servo isntance 

## Motor Actuation

In [ ]:
def rotate(pin,angle):
    board.digital[pin].write(angle)
    sleep(0.015)
    # implementing rotation function 

## Geometric & Intensity-level Transformations

In [7]:
def crop(image):
    cropped = image[130:500, 150:500]
    return cropped

In [103]:
def edge(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    filt_gauss=cv2.GaussianBlur(frame,(n,n),0)
    
    kernel_v=[[1,0,-1],
        [2,0,-2],
        [1,0,-1]]

    kernel_h=[[1,2,1],
        [0,0,0],
        [-1,-2,-1]]

    sobel_v = cv2.filter2D(filt_gauss,-1, np.array(kernel_v))
    sobel_h = cv2.filter2D(filt_gauss,-1, np.array(kernel_h))
    
    abs_grad_v = cv2.convertScaleAbs(sobel_v)
    abs_grad_h = cv2.convertScaleAbs(sobel_h)
    
    sobel = cv2.addWeighted(abs_grad_v, 0.5, abs_grad_h, 0.5, 0)
    
    canny = cv2.Canny(filt_gauss,50,200)    
    cv2.imshow('fdg circles',canny)
    return canny

In [122]:
def morphological(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    threshold = 100
    gray=cv2.GaussianBlur(gray, (n,n),0)
    binary=cv2.inRange(gray,150,255)
    
    erode=cv2.erode(binary,kernel,iterations = 4)
    dilate=cv2.dilate(binary,kernel,iterations = 2)
    closing = cv2.morphologyEx(dilate, cv2.MORPH_CLOSE, kernel)
    opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    return opening

## Hough Transformation

In [197]:
def hough(image):
    cimg = image.copy()
#     image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#     image = edge(image)
    image = morphological(image)
#     image=cv2.inRange(image,150,255)
    image = cv2.medianBlur(image,5)
    
    try:
        circles = cv2.HoughCircles(image,cv2.HOUGH_GRADIENT,1.2,120,param1=50,param2=15,minRadius=60,maxRadius=70)
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
        # draw the outer circle
            cv2.circle(image,(i[0],i[1]),i[2],(0,255,0),2)
            print((i[0],i[1]))

            # draw the center of the circle
            cv2.circle(image,(i[0],i[1]),2,(0,0,255),3)
    except:
        pass
    
    (h, w) = image.shape[:2] #w:image-width and h:image-height
    cv2.circle(image, (w//2, h//2), 7, (255, 255, 255), -1)
    
    cv2.imshow('detected circles',image)


## Main Loop

In [198]:
camera = cv2.VideoCapture(-1)
orangeLower = (18, 40, 90)
orangeUpper = (29, 255, 255)
while True:
    _, frame = camera.read()

    if frame is None:
        break
        
    midpoint = frame[479,320]
    midpoint = [0,0,255]
    
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    width, height = frame.shape[:2]
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, orangeLower, orangeUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None

    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        X = int(M["m10"] / M["m00"])
        Y = int(M["m01"] / M["m00"])
        center = (X,Y)
        print( center)
        
        Xbar = 480 - X
        Ybar = Y - 280
        theta = math.atan(Xbar/Ybar)
        if y < 280:
            angle = 90 - int(theta*(180/math.pi))
            rotate(pin,angle-20)
        else :
            angle = 90 + int(theta*(180/math.pi))
            rotate(pin,angle)
        
        print(angle)

        # To see the centroid clearly
        if radius > 10:
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 5)
            cv2.imwrite("circled_frame.png", cv2.resize(frame, (int(height / 2), int(width / 2))))
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
    
    
    cv2.imshow("Circles", frame)  
    if cv2.waitKey(1) == 13: 
        break

camera.release()
cv2.destroyAllWindows()

(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(659, 169)
(746, 385)
(746, 385)
(746, 385)
(746, 385)
(781, 379)
(746, 385)
(781, 377)
(746, 385)
(746, 385)
(746, 385)
(781, 379)
(659, 163)
(746, 385)
(746, 385)
(749, 385)
(781, 379)
(749, 385)
(746, 385)
(781, 379)
(781, 379)
(746, 385)
(781, 377)
(781, 379)
(781, 377)
(781, 379)
(781, 379)
(746, 385)
(746, 385)
(746, 385)
(781, 379)
(746, 385)
(781, 379)
(746, 385)
(781, 379)
(781, 379)
(751, 381)
(749, 377)
(637, 373)
(757, 409)
(655, 373)
(661, 365)
(631, 359)
(631, 359)
(451, 359)
(661, 401)
(667, 397)
(667, 397)
(656, 398)
(665, 403)
(659, 403)
(661, 403)
(653, 397)
(659, 401)
(661, 403)
(649, 398)
(649, 395)
(655, 397)
(655, 397)
(649, 395)
(649, 395)
(685, 433)
(694, 434)
(653, 397)
(653, 397)
(653, 397)
(653, 397)
(715, 359)
(709, 359)
(709, 365)
(709, 361)
(673, 365)
(661, 365)
(640, 362)

(641, 399)
(640, 400)
(640, 400)
(640, 400)
(640, 398)
(641, 399)
(641, 399)
(640, 398)
(641, 399)
(641, 399)
(641, 399)
(641, 399)
(641, 399)
(641, 399)
(641, 399)
(640, 400)
(640, 398)
(644, 399)
(640, 400)
(641, 399)
(643, 397)
(640, 400)
(637, 397)
(640, 398)
(640, 398)
(641, 399)
(641, 399)
(640, 400)
(640, 400)
(640, 400)
(641, 399)
(640, 400)
(641, 399)
(641, 399)
(641, 399)
(641, 399)
(641, 399)
(640, 400)
(637, 395)
(641, 399)
(640, 400)
(640, 400)
(641, 399)
(640, 398)
(640, 398)
(637, 397)
(640, 398)
(641, 399)
(640, 400)
(641, 399)
(641, 399)
(640, 398)
(640, 400)
(641, 399)
(640, 400)
(640, 400)
(640, 400)
(640, 400)
(640, 400)
(640, 400)
(637, 397)
(640, 400)
(640, 400)
(640, 400)
(640, 400)
(640, 400)
(640, 398)
(641, 399)
(641, 399)
(641, 399)
(641, 399)
(641, 399)
(641, 399)
(637, 397)
(640, 400)
(641, 399)
(640, 398)
(640, 398)
(640, 398)
(641, 399)
(641, 399)
(645, 399)
(641, 399)
(641, 399)
(641, 399)
(640, 400)
(641, 399)
(641, 397)
(640, 398)
(640, 400)
(641, 399)

(639, 394)
(641, 394)
(643, 400)
(643, 397)
(643, 395)
(641, 394)
(641, 398)
(644, 398)
(640, 398)
(641, 399)
(641, 397)
(641, 394)
(637, 397)
(641, 394)
(637, 395)
(643, 400)
(643, 395)
(641, 394)
(640, 398)
(643, 394)
(641, 394)
(637, 395)
(641, 397)
(641, 399)
(643, 397)
(641, 398)
(641, 398)
(641, 394)
(640, 398)
(640, 398)
(640, 398)
(640, 398)
(640, 398)
(640, 397)
(640, 398)
(645, 398)
(640, 394)
(637, 395)
(641, 398)
(637, 395)
(641, 399)
(641, 397)
(637, 395)
(640, 397)
(641, 394)
(637, 395)
(638, 393)
(637, 395)
(640, 397)
(637, 395)
(641, 397)
(641, 398)
(637, 395)
(641, 397)
(640, 394)
(641, 397)
(638, 393)
(641, 397)
(641, 397)
(640, 394)
(637, 395)
(640, 395)
(640, 397)
(643, 398)
(641, 398)
(637, 395)
(640, 394)
(641, 398)
(641, 397)
(640, 394)
(637, 395)
(637, 395)
(637, 395)
(641, 394)
(641, 398)
(641, 394)
(637, 395)
(641, 398)
(641, 397)
(641, 398)
(641, 398)
(641, 397)
(637, 393)
(637, 395)
(640, 394)
(641, 398)
(637, 393)
(637, 393)
(637, 395)
(640, 400)
(639, 394)

(637, 393)
(640, 400)
(641, 398)
(641, 398)
(641, 398)
(640, 397)
(638, 392)
(640, 397)
(640, 400)
(641, 397)
(637, 393)
(640, 397)
(640, 397)
(640, 397)
(640, 397)
(640, 397)
(645, 398)
(640, 400)
(640, 397)
(641, 395)
(640, 397)
(640, 397)
(640, 397)
(637, 393)
(637, 393)
(640, 397)
(643, 398)
(517, 559)
(640, 400)
(640, 397)
(641, 398)
(641, 395)
(638, 392)
(643, 397)
(637, 393)
(640, 397)
(638, 393)
(640, 397)
(643, 398)
(637, 393)
(640, 397)
(637, 395)
(640, 397)
(637, 393)
(517, 559)
(637, 395)
(640, 397)
(637, 395)
(637, 395)
(637, 395)
(641, 395)
(640, 397)
(637, 393)
(640, 400)
(640, 397)
(640, 397)
(640, 400)
(637, 395)
(637, 393)
(638, 393)
(637, 395)
(640, 397)
(640, 397)
(637, 393)
(641, 395)
(641, 399)
(637, 393)
(637, 395)
(640, 397)
(641, 395)
(643, 398)
(643, 398)
(640, 400)
(637, 395)
(639, 394)
(637, 393)
(640, 397)
(637, 393)
(639, 394)
(637, 393)
(637, 393)
(637, 395)
(640, 397)
(640, 397)
(637, 393)
(641, 399)
(641, 398)
(641, 399)
(640, 397)
(640, 397)
(640, 397)

(639, 394)
(640, 395)
(637, 395)
(641, 398)
(637, 395)
(640, 397)
(638, 397)
(638, 397)
(640, 395)
(517, 559)
(641, 398)
(638, 397)
(638, 397)
(637, 395)
(641, 398)
(638, 397)
(637, 393)
(641, 398)
(640, 397)
(638, 397)
(640, 397)
(641, 398)
(641, 398)
(638, 397)
(639, 394)
(637, 393)
(641, 398)
(640, 397)
(640, 395)
(640, 395)
(641, 398)
(641, 398)
(641, 398)
(640, 397)
(641, 398)
(638, 397)
(638, 397)
(638, 397)
(640, 397)
(641, 398)
(640, 397)
(641, 398)
(637, 395)
(641, 397)
(637, 395)
(641, 398)
(640, 395)
(640, 397)
(638, 397)
(637, 395)
(641, 400)
(637, 393)
(641, 398)
(638, 397)
(638, 397)
(637, 395)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(640, 395)
(641, 398)
(637, 393)
(638, 397)
(638, 397)
(637, 395)
(638, 397)
(641, 398)
(641, 398)
(638, 397)
(638, 397)
(638, 397)
(641, 398)
(641, 398)
(640, 395)
(638, 397)
(643, 397)
(641, 398)
(641, 397)
(637, 395)
(640, 397)
(641, 398)
(641, 394)
(640, 395)
(638, 397)
(641, 398)
(638, 397)
(641, 398)
(640, 397)
(640, 395)
(641, 398)

(638, 397)
(638, 397)
(638, 397)
(641, 398)
(643, 399)
(638, 395)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 401)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(640, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(640, 398)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(643, 397)
(517, 559)
(638, 397)
(638, 395)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(641, 398)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(641, 398)
(638, 397)
(640, 397)
(638, 397)
(638, 397)
(638, 397)
(515, 559)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(638, 397)
(505, 509)
(638, 401)
(639, 394)
(638, 397)
(637, 393)
(638, 397)
(638, 397)
(641, 398)
(638, 397)
(643, 397)
(643, 397)
(638, 397)
(641, 398)
(638, 397)
(639, 394)
(641, 398)
(638, 397)
(641, 398)
(643, 397)

(638, 394)
(640, 398)
(640, 395)
(641, 397)
(638, 394)
(641, 399)
(638, 394)
(638, 394)
(640, 397)
(637, 395)
(640, 397)
(640, 397)
(640, 397)
(637, 392)
(638, 394)
(640, 398)
(638, 394)
(640, 397)
(638, 394)
(639, 394)
(638, 394)
(640, 397)
(640, 395)
(640, 395)
(638, 394)
(638, 394)
(637, 392)
(638, 394)
(640, 397)
(638, 394)
(641, 397)
(640, 397)
(637, 395)
(640, 393)
(638, 394)
(640, 397)
(639, 394)
(637, 395)
(638, 394)
(638, 394)
(640, 398)
(640, 397)
(639, 394)
(640, 398)
(639, 394)
(638, 394)
(640, 397)
(638, 400)
(640, 395)
(638, 394)
(638, 400)
(637, 395)
(638, 394)
(640, 393)
(639, 391)
(640, 397)
(638, 394)
(640, 397)
(639, 394)
(638, 394)
(638, 394)
(638, 394)
(639, 394)
(638, 394)
(638, 394)
(638, 394)
(638, 394)
(638, 394)
(639, 394)
(637, 395)
(638, 394)
(638, 394)
(640, 397)
(640, 397)
(638, 394)
(640, 393)
(640, 397)
(641, 397)
(641, 397)
(640, 397)
(640, 397)
(640, 395)
(639, 394)
(637, 395)
(638, 394)
(638, 394)
(637, 395)
(638, 394)
(638, 394)
(638, 394)
(640, 397)